In [54]:
import pandas as pd
from geopy.distance import geodesic
from scipy.stats import zscore
import math
import os

In [55]:
def calculate_bearing(row):
    lat1 = math.radians(row["previous_latitude"])
    lon1 = math.radians(row["previous_longitude"])
    lat2 = math.radians(row["latitude"])
    lon2 = math.radians(row["longitude"])

    delta_lon = lon2 - lon1

    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
                                           * math.cos(lat2) *
                                           math.cos(delta_lon))

    initial_bearing = math.atan2(x, y)
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = int((initial_bearing + 360) % 360)

    return compass_bearing


def calculate_distance(row):
    return geodesic((row["previous_latitude"], row["previous_longitude"]),
                    (row["latitude"], row["longitude"])).meters


def prepare_data(df):
    df["previous_latitude"] = df["latitude"].shift(1)
    df["previous_longitude"] = df["longitude"].shift(1)
    df["distance_from_takeoff_m"] = df.apply(
        lambda row: geodesic(
            (row["latitude"], row["longitude"]),
            (df["latitude"].iloc[0], df["longitude"].iloc[0])).meters, axis=1)
    df.drop(df.index[0], inplace=True)
    df["distance_m"] = df.apply(lambda row: geodesic(
        (row["previous_latitude"], row["previous_longitude"]),
        (row["latitude"], row["longitude"])).meters, axis=1)
    df["speed_km/s"] = ((df["distance_m"]/1000) /
                        (df["datetime"].diff().dt.total_seconds()/3600))
    df["climb_m"] = df["gps_altitude_m"].diff()
    df["climb_m(delta)"] = df["gps_altitude_m"].diff(20)
    df["climb_rate_m/s"] = df["climb_m"] / \
        df["datetime"].diff().dt.total_seconds()
    df["bearing"] = df.apply(calculate_bearing, axis=1)
    df["delta_bearing"] = abs((df["bearing"].diff() + 180) % 360 - 180)
    df["glide_ratio"] = df.apply(
        lambda row: row["distance_m"] /
        abs(row["climb_m"]) if row["climb_m"] != 0 else 0,
        axis=1)
    df.fillna(0, inplace=True)
    # find and delete before take off
    # for index, row in df.iterrows():
    #     if row["speed_km/s"] > 10:
    #         df = df.iloc[index:]
    #         df.reset_index(drop=True, inplace=True)
    #         break
    df["elapsed_time"] = (
        df["datetime"] - df["datetime"].iloc[0]).dt.total_seconds()
    df["zone"] = df.apply(lambda row: "thermal" if (
        row["climb_m(delta)"] > 0 or
        row["climb_rate_m/s"] > 0)
        else "standart", axis=1)
    df = df[df["zone"] == "thermal"]
    return df


In [56]:
df = pd.read_csv(os.path.join("data", "flight_data.csv"), low_memory=False)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344002 entries, 0 to 2344001
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   filename             object 
 1   datetime             object 
 2   pilot                object 
 3   latitude             float64
 4   longitude            float64
 5   gps_altitude_m       int64  
 6   pressure_altitude_m  int64  
 7   temp                 float64
 8   pressure             float64
 9   humidity             float64
 10  dew_point            float64
 11  wind_speed           float64
 12  wind_deg             float64
dtypes: float64(8), int64(2), object(3)
memory usage: 232.5+ MB


,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,file_69489_2020-07-03-XCT-XXX-02.igc,2020-07-03 07:42:07,NaN,36.980733,29.31435,2087,2130,25.62,1008.0,35.0,9.04,0.66,75.0
1,file_69489_2020-07-03-XCT-XXX-02.igc,2020-07-03 07:42:08,NaN,36.980733,29.31435,2087,2130,25.62,1008.0,35.0,9.04,0.66,75.0
2,file_69489_2020-07-03-XCT-XXX-02.igc,2020-07-03 07:42:09,NaN,36.980733,29.31435,2087,2130,25.62,1008.0,35.0,9.04,0.66,75.0
3,file_69489_2020-07-03-XCT-XXX-02.igc,2020-07-03 07:42:10,NaN,36.980733,29.31435,2087,2130,25.62,1008.0,35.0,9.04,0.66,75.0
4,file_69489_2020-07-03-XCT-XXX-02.igc,2020-07-03 07:42:11,NaN,36.980733,29.31435,2087,2130,25.62,1008.0,35.0,9.04,0.66,75.0


In [57]:
df.describe()

,latitude,longitude,gps_altitude_m,pressure_altitude_m,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06,2.344002e+06
mean,3.843023e+01,3.196094e+01,2.175270e+03,2.623137e+03,3.078743e+01,1.007535e+03,2.687358e+01,8.988506e+00,3.196908e+00,2.131267e+02
std,1.048362e+00,2.631969e+00,2.018055e+03,7.256296e+02,3.560259e+00,2.609563e+00,7.290759e+00,3.105485e+00,1.352502e+00,1.200057e+02
min,3.650395e+01,2.463248e+01,-7.347000e+03,0.000000e+00,1.838000e+01,9.990000e+02,6.000000e+00,-5.280000e+00,0.000000e+00,0.000000e+00
25%,3.746465e+01,3.003208e+01,1.128000e+03,2.120000e+03,2.840000e+01,1.006000e+03,2.200000e+01,7.000000e+00,2.190000e+00,1.090000e+02
50%,3.838428e+01,3.200002e+01,2.274000e+03,2.623000e+03,3.051000e+01,1.007000e+03,2.700000e+01,9.280000e+00,3.220000e+00,2.410000e+02
75%,3.918852e+01,3.267773e+01,2.859000e+03,3.107000e+03,3.313000e+01,1.009000e+03,3.200000e+01,1.111000e+01,4.060000e+00,3.170000e+02
max,4.272443e+01,4.104910e+01,1.619900e+04,9.999000e+03,4.111000e+01,1.018000e+03,5.800000e+01,2.225000e+01,8.700000e+00,3.600000e+02


In [58]:
# df["gps_altitude_m"] = df.apply(lambda row: row["gps_altitude_m"] if row["gps_altitude_m"] < 6000 else row["pressure_altitude_m"], axis=1)
# df["pressure_altitude_m"] = df.apply(lambda row: row["pressure_altitude_m"] if row["pressure_altitude_m"] < 6000 else row["gps_altitude_m"], axis=1)
df["gpszscore"] = zscore(df["gps_altitude_m"])
df["presurezscore"] = zscore(df["pressure_altitude_m"])
df = df[df["gpszscore"].abs() < 3]
df.describe()

,latitude,longitude,gps_altitude_m,pressure_altitude_m,temp,pressure,humidity,dew_point,wind_speed,wind_deg,gpszscore,presurezscore
count,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06,2.274436e+06
mean,3.840892e+01,3.193330e+01,1.896002e+03,2.633565e+03,3.087671e+01,1.007431e+03,2.675036e+01,8.988623e+00,3.155061e+00,2.112280e+02,-1.383850e-01,1.437084e-02
std,1.051894e+00,2.666110e+00,1.222744e+03,7.279487e+02,3.538940e+00,2.526291e+00,7.318322e+00,3.092988e+00,1.339419e+00,1.196050e+02,6.059026e-01,1.003196e+00
min,3.650395e+01,2.463248e+01,-3.602000e+03,0.000000e+00,1.838000e+01,9.990000e+02,6.000000e+00,-5.280000e+00,0.000000e+00,0.000000e+00,-2.862792e+00,-3.614982e+00
25%,3.743760e+01,2.999637e+01,1.028000e+03,2.126000e+03,2.845000e+01,1.006000e+03,2.200000e+01,7.030000e+00,2.140000e+00,9.500000e+01,-5.685032e-01,-6.851120e-01
50%,3.835100e+01,3.176914e+01,2.235000e+03,2.635000e+03,3.060000e+01,1.007000e+03,2.700000e+01,9.280000e+00,3.160000e+00,2.400000e+02,2.959773e-02,1.634798e-02
75%,3.915744e+01,3.263468e+01,2.809000e+03,3.123000e+03,3.322000e+01,1.009000e+03,3.200000e+01,1.111000e+01,4.010000e+00,3.150000e+02,3.140301e-01,6.888675e-01
max,4.272443e+01,4.104910e+01,4.776000e+03,9.999000e+03,4.111000e+01,1.018000e+03,5.800000e+01,2.225000e+01,8.700000e+00,3.600000e+02,1.288731e+00,1.016478e+01
